I am trying to do the following steps in pyspark notebook in MS Fabric: 
Step 1: I am doing the following text
Step 2: Create Reference DF for Frequency
Step 3: Determining Frequency for each Semantic_Model_ID
Step 4: Save Spark DF to Delta Table
CAn you please optimize the code below with debugging messages at proper sections.
Generate and ipynb notebook for this

In [ ]:
# Header_Files
import requests
import json
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from notebookutils import mssparkutils
print("All Libraries Loaded")

# Get connections
client_id = mssparkutils.credentials.getSecret("https://kv-pbi-techical-accounts.vault.azure.net/", "clientid")
client_secret = mssparkutils.credentials.getSecret("https://kv-pbi-techical-accounts.vault.azure.net/", "clientsecret")
tenant_id = mssparkutils.credentials.getSecret("https://kv-pbi-techical-accounts.vault.azure.net/", "tenantid")
dataset_id = mssparkutils.credentials.getSecret("https://kv-pbi-techical-accounts.vault.azure.net/", "datasetid-fabric-capacity-metrics")
resource = 'https://analysis.windows.net/powerbi/api'
api_version = 'v1.0'
refresh_schedule_path = 'abfss://14b44c0a-e3c3-41eb-b31f-c2a7d90ce593@onelake.dfs.fabric.microsoft.com/e6e52507-e786-469c-8048-c816d44f5b5a/Tables/mard_mdna_t_pbi_sm_refresh_schedule'

# Get access token
auth_url = 'https://login.microsoftonline.com/{0}/oauth2/token'.format(tenant_id)
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'resource': resource
}
auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json()['access_token']

headers = {
    'Authorization': 'Bearer {0}'.format(access_token)
}

# Step 1: Get Refresh Schedule delta table
df_spark_refresh_schedule = spark.read.format("delta").load(refresh_schedule_path)

df_refresh_schedule = df_spark_refresh_schedule.toPandas()

print(df_refresh_schedule.head())
print(df_refresh_schedule.info())


# Step 2: Create Reference DF for Frequency
df_refresh_frequency = pd.DataFrame()
df_refresh_frequency['Key_SM_Refresh_Day'] = None
df_refresh_frequency['Semantic_Model_ID'] = None
df_refresh_frequency['Refresh_Day'] = None

df_refresh_frequency['Key_SM_Refresh_Day'] = df_refresh_schedule.apply(
    lambda row: 
    row['Semantic_Model_ID'] + "_" 
    + str(row['Time_of_the_Day']) + "_"
    + row['Time_zone'] 
, axis=1)

df_refresh_frequency['Refresh_Day'] = df_refresh_schedule.apply(
    lambda row: 
    row['Day']
, axis=1)

df_refresh_frequency['Semantic_Model_ID'] = df_refresh_schedule.apply(
    lambda row: 
    row['Semantic_Model_ID']
, axis=1)
print(df_refresh_frequency.head())
print(df_refresh_frequency.info())

# Step 3: Determining Frequency for each Semantic_Model_ID
##Loading the Keys in a List for iteration

distinct_keys = df_refresh_frequency['Key_SM_Refresh_Day'].unique().tolist()
print(f"Length of Keys is: {len(distinct_keys)}")
list_frequency = []

# Define the set of all days in a week
all_days = {"Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"}

# Iterate over the distinct_keys list
for key1 in distinct_keys:
    # Filter the dataframe for rows where 'Key' matches the current key
    df_temp = df_refresh_frequency[df_refresh_frequency['Key_SM_Refresh_Day'] == key1]
    
    # Get the unique Semantic_Model_ID from df_temp (assuming it's the same for all rows)
    semantic_model_id = df_temp['Semantic_Model_ID'].iloc[0]
    
    # Get the unique days from the Refresh_Day column
    refresh_days = set(df_temp['Refresh_Day'].unique())
    
    # Determine the frequency based on the days present
    if refresh_days == all_days:
        frequency = "Daily"
    else:
        frequency = "Weekly"
    
    # Append the result to the final_results list
    list_frequency.append(
        {"Key_Frequency": semantic_model_id + "_" + frequency, 
        "Semantic_Model_ID": semantic_model_id, 
        "Frequency": frequency}
    )

# Create the final dataframe from the results
df_frequency = pd.DataFrame(list_frequency)

# Remove duplicate rows from the final dataframe
df_frequency = df_frequency.drop_duplicates()

# Print the final dataframe
print(df_frequency)

# Inner Join with df_refresh_schedule
df_merged = pd.merge(df_frequency, df_refresh_schedule, on='Semantic_Model_ID', how='inner')
print(df_merged.info())

# Step 4: Save Spark DF to Delta Table
df_Key_destination = spark.sql("SELECT DISTINCT Key_Frequency FROM mdna_pbi_monitoring.MARD_MDNA_T_PBI_SM_Refresh_Schedule_Frequency").collect()

destination_keys = {row['Key_Frequency'] for row in df_Key_destination}

loaded_rows = 0
not_loaded_rows = 0
filtered_rows = []
unfiltered_rows = []

# Compare Key_Frequency from df_merged with Key_Frequency from mdna_pbi_monitoring.MARD_MDNA_T_PBI_SM_Refresh_Schedule_Frequency Table
for index, row in df_merged.iterrows():
    df_Key = row['Key_Frequency']
    if df_Key not in destination_keys:
        filtered_rows.append(row)
        loaded_rows += 1
    else:
        not_loaded_rows += 1
        unfiltered_rows.append(row)

print("No. of Loaded Rows: {0} \nNo. of Not Loaded Rows: {1}".format(loaded_rows, not_loaded_rows))

filtered_df = pd.DataFrame(filtered_rows) # New records 
unfiltered_df = pd.DataFrame(unfiltered_rows) # Existing records


# If new Data is there append to the the database
if loaded_rows > 0: 
    df_spark_rowsfiltered = spark.createDataFrame(filtered_df)
    df_spark_rowsfiltered.write.format("delta").mode("append").saveAsTable("MARD_MDNA_T_PBI_SM_Refresh_Schedule_Frequency")

# Post-load checking
df_count = spark.sql("SELECT count(*) FROM mdna_pbi_monitoring.MARD_MDNA_T_PBI_SM_Refresh_Schedule_Frequency")
display(df_count)